In [1]:
import requests
import pandas as pd
import numpy as np
import smtplib

In [2]:
buying_Price = int(input("Input buying price in INR for BTC: "))

In [3]:
Selling_Price = int(input("What should be your selling price for BTC: "))

In [4]:
Volume = input("Enter the Volume of the BTC-INR: ")
Volume = float(Volume)

In [5]:
final_buying_price = (buying_Price)*Volume

In [6]:
final_selling_price = (Selling_Price)*Volume

In [7]:
makers_fee = 0.002*(buying_Price)*(Volume)


In [8]:
takers_fee = 0.002*(Selling_Price)*(Volume)

In [9]:
Final_fee = makers_fee+takers_fee

In [10]:
Profit_inc = (Selling_Price - buying_Price)*Volume
Profit_per = (Profit_inc/buying_Price)*100


In [11]:
Profit_Final = Profit_inc - Final_fee

In [12]:
url = "https://api.wazirx.com/api/v2/trades?market=btcinr"

In [13]:
response = requests.get(url)

In [14]:
response.status_code

200

In [15]:
data = response.json()[1]

In [16]:
response = requests.get(url)
data = response.json()[1]

In [17]:
with open("BTC_INR.csv","a+") as fp:
    fp.write(f"\n{data['market']},{buying_Price},{Selling_Price},{data['price']},{Profit_inc},{Profit_Final},{data['created_at']},{data['volume']},{data['funds']},{data['id']}")


In [18]:
sender_email = "SENDER_EMAIL"
rec_mail = "RECEIVER_EMAIL"
password = "PASSWORD"

In [19]:
server = smtplib.SMTP('smtp.gmail.com',587)
server.starttls()
server.login(sender_email,password)

(235, b'2.7.0 Accepted')

In [22]:
while True:
    response = requests.get(url)
    data = response.json()[1]
    df = pd.read_csv('BTC_INR.csv', index_col=0)
    cur_Profit_inc = (float(data['price']) - buying_Price)*Volume
    cur_Profit_per = (Profit_inc/buying_Price)*100
    cur_makers_fee = 0.002*(buying_Price)*(Volume)
    cur_takers_fee = 0.002*(float(data['price']))*(Volume)
    cur_Final_fee = cur_makers_fee + cur_takers_fee
    cur_Profit_Final = cur_Profit_inc - cur_Final_fee
    if (data["id"] !=  df.iloc[-1,-1]):
        with open("BTC_INR.csv","a+") as fp:
            fp.seek(0,0)
            if (float(data['price'])>= 0.98*(Selling_Price)):
                message = f"""
                The price is close to your desired Selling Price
                Current Price: {float(data['price'])}
                Current Profit: {cur_Profit_inc}
                Current Final Fee: {cur_Final_fee}
                Current Profit Final: {cur_Profit_Final}
                """
                server.sendmail(sender_email,rec_mail,message)
            if (float(data['price']) <= 1.02*(buying_Price)):
                message = f"""
                The price is close to your desired Buying Price
                Current Price: {float(data['price'])}
                Current Final Fee: {0.002*(float(data['price']))*(Volume)}
                """
                server.sendmail(sender_email,rec_mail,message)
            fp.write(f"\n{data['market']},{buying_Price},{Selling_Price},{data['price']},{cur_Profit_inc},{cur_Profit_Final},{data['created_at']},{data['volume']},{data['funds']},{data['id']}")

KeyboardInterrupt: 